In [5]:
import numpy as np
import pickle
import torch
import random
import os
from model_bet import Graph_Diff_Reg
from sklearn.metrics import r2_score
import time

os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
def edge_list_to_adjacency_matrix(edge_list, edge_weights, num_nodes):
    # Initialize an empty adjacency matrix with zeros
    adj_matrix = np.zeros((num_nodes, num_nodes))

    # Iterate through the edge list and fill in the adjacency matrix
    for edge, weight in zip(edge_list, edge_weights):
        source, target = edge
        adj_matrix[source][target] = weight

    return adj_matrix

In [ ]:
def train(filefolder):
    model.train() 
    
    loss_train = 0
    num_files_train = len(filefolder)
    
    starttime3 = time.time()
    y_trues=[]
    y_preds=[]
    
    random.shuffle(filefolder)
    n=0
    
    for i in range(num_files_train):
        print("Batch: "+str(i))

        filepath=filefolder[i]
        with open(filepath,"rb") as f:
            # list_adj_deg_train_shuffled, list_adj_wt_train_shuffled,list_num_edge_train, bc_mat_train_shuffled, list_fm_train_shuffled, list_n_seq_train, list_n_unshuffle_train,list_adj_deg_test_shuffled, list_adj_wt_test_shuffled,list_num_edge_test, bc_mat_test_shuffled, list_fm_test_shuffled, list_n_seq_test, list_n_unshuffle_test=pickle.load(f)
            list_train_graph, list_edge_index_train,list_edge_weight_train, eff_mat_train, list_fm_train_orig,list_edge_index_train_orig,list_edge_weight_train_orig=pickle.load(f)
        
        # for j in range(len(list_train_graph)):
        #     print(len(list_train_graph[j].edges()))
        
        
        for j in range(len(list_train_graph)):
            
            # g=list_train_graph
            # adj2=torch.tensor(nx.adjacency_matrix(list_train_graph[j]).toarray()).numpy()
            
            # for 
            
            
            edge_index=list_edge_index_train[j]
            edge_weigth=list_edge_weight_train[j]
            
            adj=torch.tensor(edge_list_to_adjacency_matrix(edge_index.numpy().transpose(), edge_weigth.numpy(), 500))
            adj=adj.to(device)
            
            edge_index=edge_index.to(device)
            edge_weigth=edge_weigth.to(device)
            fm=list_fm_train_orig[0]
            # fm=torch.tensor(np.eye(500,dtype=np.float32))
            
            batch_vector=np.zeros((fm.shape[0]),dtype=np.int64)
            batch_vector=torch.tensor(batch_vector)
            batch_vector=batch_vector.to(device)
            
            fm=fm.to(device).to_dense()
            # adj_orig=torch.tensor(nx.adjacency_matrix(list_train_graph_orig[j]).toarray())
            edge_index_orig=list_edge_index_train_orig[0]
            edge_weigth_orig=list_edge_weight_train_orig[0]
            
            adj_orig=torch.tensor(edge_list_to_adjacency_matrix(edge_index_orig.numpy().transpose(), edge_weigth_orig.numpy(), 500))
            adj_orig=adj_orig.to(device)
            
            adjn=adj.cpu().numpy()
            adjon=adj_orig.cpu().numpy()
            adjr=adjon-adjn
            adjr[adjr!=0]=1
            adjr=torch.tensor(adjr.astype(np.float32)).to(device)
            
            # edge_index_n=edge_index.cpu().numpy()
            # edge_index_orig_n=edge_index_orig.cpu().numpy()
            edge_weight_n=edge_weigth.cpu().numpy()
            edge_weight_n[edge_weight_n<10000]=1
            edge_weight_n[edge_weight_n!=1]=0
            edge_mask=torch.tensor(edge_weight_n.astype(np.int64)).to(device)
            
            edge_index_orig=edge_index_orig.to(device)
            
            edge_weigth_orig=edge_weigth_orig.to(device)
            
            y_true=torch.tensor(eff_mat_train[0,j].reshape([1,1]).astype(np.float32))
            y_true=y_true.to(device)
            
            # adj = list_adj_deg_train_shuffled[j]
            # num_nodes = list_num_edge_train[j]
            # adj_t = list_adj_wt_train_shuffled[j]
            # fm = list_fm_train_shuffled[j]
            # adj = adj.to(device)
            # adj_t = adj_t.to(device)
            # fm=fm.to(device)
            
            optimizer.zero_grad()
                
            # y_out = model(adj,adj_t,fm)
            y_out = model(edge_index,edge_weigth.float(),edge_index_orig,edge_weigth_orig.float(),fm.float(),batch_vector,adjr.float(),edge_mask)#,adj,adj_orig)
                        
            # true_arr = torch.from_numpy(bc_mat_train_shuffled[:,j]).float()
            # true_val = true_arr.to(device)
            
            # true_val_final = true_val[list_n_unshuffle_train[j]]
            # y_out_final = y_out[list_n_unshuffle_train[j]]
            
            # true_val_ultimate = true_val_final[:num_nodes]
            # y_out_ultimate = y_out_final[:num_nodes]
            
            mse = loss_fn(y_out, y_true)
            
            mse.backward()
            
            
            
            optimizer.step()
            
            y_trues.append(float(y_true))
            y_preds.append(float(y_out))
            
            
            # loss_rank = new_loss_cal(y_out_ultimate, true_val_ultimate, num_nodes, device, model_size)
            loss_train = loss_train + float(mse)
            n+=1
            
    mse_final=loss_train/n
    duration3 = time.time()-starttime3
    print("training time this epoch:"+str(duration3))
    print("training loss:"+str(loss_train))
    print("training rmse:"+str(np.sqrt(float(mse_final))))
    
    score=r2_score(y_trues, y_preds)
    print("Training_R_Squared:"+str(score))
    
    return loss_train,mse_final,score

In [ ]:
hidden = 256
num_epoch = 200
n_features = 256
dropout_val = 0.4
learning_rate = 0.00005

model = Graph_Diff_Reg(
    input_dim=n_features,
    hidden_dim=hidden,
    output_dim=1,
    dropout=dropout_val
)

print(f'The following device is used {device}')
model.to(device)

loss_fn = torch.nn.MSELoss()  # mean square error
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=0.00005/10)

training_loss_epochs = []
training_mse_epochs = []

testing_loss_epochs= []
testing_mse_epochs= []

training_score_epochs = []
testing_score_epochs = []